# WVS Wave 7 Data Analysis - EFA Model Comparison

This script compares different Exploratory Factor Analysis (EFA) models by iterating through various extraction methods and rotation types for a specified number of factors. It calculates metrics like explained variance, RMSR, simple structure count, factor correlations, and model fit indices (for ML) to help identify the best-fitting model configuration based on the aggregated data.


In [16]:
# =============================================================================
# Imports
# =============================================================================
from __future__ import annotations
import argparse
import datetime as dt
import os
import sys
import warnings
import pathlib

import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
# NOTE: Imputation imports removed - assuming aggregated data is complete.


In [ ]:
# =============================================================================
# Configuration / Constants
# =============================================================================
DATA_DIR = pathlib.Path("./data")
OUTPUT_DIR_ROOT = pathlib.Path("./output/efa_comparison")

# --- Input Data ---
# AGGREGATED_DATA_PATH = DATA_DIR / "wvs_wave7_aggregated_by_demographics.csv"
AGGREGATED_DATA_PATH = DATA_DIR / "new_median_wvs_wave7_aggregated_by_demographics.csv"
# File containing variable codes (needed to identify survey columns)
VARIABLE_INFO_PATH = DATA_DIR / "variable_info.csv" # Ensure this matches previous scripts

# --- EFA Comparison Parameters ---
RANDOM_STATE = 42
DEFAULT_N_FACTORS = 5 # Example: Number of factors to compare models for
DEFAULT_LOADING_THRESHOLD = 0.40 # For simple structure calculation
METHODS_TO_COMPARE = ["minres", "principal", "ml"] # EFA extraction methods
ROTATIONS_TO_COMPARE = ["promax", "oblimin", "varimax"] # Rotation types

In [ ]:
# =============================================================================
# Helper Functions
# =============================================================================

def ensure_output_dir(root: pathlib.Path, prefix: str = "") -> pathlib.Path:
    """Creates a date-stamped output directory."""
    today = dt.date.today().isoformat().replace("-", "")
    dir_name = f"{prefix}{today}" if prefix else today
    out_path = root / dir_name
    out_path.mkdir(parents=True, exist_ok=True)
    print(f"Output directory created/ensured: {out_path}")
    return out_path

def log_message(msg: str) -> None:
    """Prints a message to the console."""
    print(msg, flush=True)

def calculate_rmsr(observed_corr: np.ndarray, reproduced_corr: np.ndarray) -> float:
    """Root Mean Square Residual of off-diagonal elements."""
    if observed_corr.shape != reproduced_corr.shape:
        raise ValueError("Observed and reproduced correlation matrices must have the same shape.")
    residual = observed_corr - reproduced_corr
    # Get lower triangle indices (excluding diagonal k=-1)
    lower_triangle_indices = np.tril_indices_from(residual, k=-1)
    squared_residuals = residual[lower_triangle_indices] ** 2
    rmsr_value = np.sqrt(np.mean(squared_residuals))
    return float(rmsr_value)

def count_simple_structure(loadings: pd.DataFrame, threshold: float) -> int:
    """Counts variables loading significantly onto exactly one factor."""
    if threshold <= 0:
        return 0 # Threshold must be positive

    # Create a boolean mask where absolute loading >= threshold
    significant_mask = loadings.abs() >= threshold

    # Count how many factors each variable loads significantly onto
    loadings_per_variable = significant_mask.sum(axis=1)

    # Count how many variables load onto exactly one factor
    simple_structure_count = int((loadings_per_variable == 1).sum())
    return simple_structure_count

def load_data_for_comparison(aggregated_path: pathlib.Path, variable_info_path: pathlib.Path) -> pd.DataFrame:
    """Loads aggregated data and identifies survey columns for EFA."""
    log_message("--- Loading Data for Comparison ---")
    # Load aggregated data
    if not aggregated_path.exists():
        raise FileNotFoundError(f"Aggregated data file not found: {aggregated_path}")
    try:
        df_agg = pd.read_csv(aggregated_path)
        log_message(f"Loaded aggregated data: {df_agg.shape}")
    except Exception as e:
        raise IOError(f"Error reading aggregated data file '{aggregated_path}': {e}")

    # Identify survey columns using variable info file
    survey_cols = []
    if variable_info_path.exists():
        try:
            var_info_df = pd.read_csv(variable_info_path)
            known_vars = var_info_df['Variable_Code'].tolist()
            survey_cols = [col for col in df_agg.columns if col in known_vars]
            log_message(f"Identified {len(survey_cols)} survey columns based on variable info file.")
        except Exception as e:
            log_message(f"Warning: Could not load/process variable info file '{variable_info_path}': {e}")
    else:
        log_message(f"Warning: Variable info file not found at '{variable_info_path}'.")

    # Fallback if survey_cols identification failed
    if not survey_cols:
        survey_cols = df_agg.select_dtypes(include=np.number).columns.tolist()
        # Exclude potential demographic columns if they are numeric by chance
        demo_cols_in_data = [col for col in AGGREGATION_DEMOGRAPHIC_VARS if col in survey_cols]
        if demo_cols_in_data:
             survey_cols = [col for col in survey_cols if col not in demo_cols_in_data]
        log_message(f"Warning: Identifying survey columns via fallback (numeric type). Found {len(survey_cols)} columns.")


    if not survey_cols:
         raise ValueError("Could not identify survey question columns for EFA.")

    df_efa_input = df_agg[survey_cols]

    # Check for NaNs
    if df_efa_input.isnull().any().any():
        log_message("Warning: Missing values found in data for EFA comparison. This might indicate issues in prior steps.")

    return df_efa_input

In [ ]:
# =============================================================================
# Core Comparison Function
# =============================================================================

def compare_efa_models(
    df_efa_input: pd.DataFrame,
    n_factors: int,
    methods: list[str],
    rotations: list[str],
    simple_structure_threshold: float
) -> pd.DataFrame:
    """Iterates through EFA models and compiles comparison metrics."""
    log_message(f"\n--- Comparing EFA Models (n_factors={n_factors}) ---")

    # --- Suitability Check ---
    try:
        chi_square_value, p_value = calculate_bartlett_sphericity(df_efa_input)
        kmo_per_variable, kmo_overall = calculate_kmo(df_efa_input)
        log_message(f"Data Suitability: Bartlett p={p_value:.4g}, Overall KMO={kmo_overall:.3f}")
        if p_value > 0.05 or kmo_overall < 0.6:
             log_message("Warning: Data suitability for EFA may be questionable based on tests.")
    except Exception as e:
        log_message(f"Warning: Could not perform suitability tests: {e}")

    # --- Iterate through Model Combinations ---
    observed_corr_matrix = df_efa_input.corr().values # Calculate once
    summary_data = []

    for method in methods:
        for rotation in rotations:
            log_message(f"Fitting Model: Method='{method}', Rotation='{rotation}'")
            try:
                fa = FactorAnalyzer(
                    n_factors=n_factors,
                    method=method,
                    rotation=rotation,
                    use_smc=True, # Use squared multiple correlation
                    rotation_kwargs={"max_iter": 1000} # For promax/oblimin
                )
                fa.fit(df_efa_input)

                # --- Calculate Metrics ---
                # Variance Explained
                _, prop_var, cum_var = fa.get_factor_variance()
                total_explained_pct = cum_var[-1] * 100 if cum_var.size > 0 else 0.0

                # RMSR (Root Mean Square Residual)
                loadings = fa.loadings_
                if hasattr(fa, 'communalities_'):
                     communalities = fa.communalities_ # Newer versions
                else:
                     communalities = (loadings**2).sum(axis=1) # Older versions
                
                # Ensure communalities have the right shape/index if needed
                reproduced_corr = loadings @ loadings.T + np.diag(communalities) # Corrected formula for reproduced correlation
                rmsr = calculate_rmsr(observed_corr_matrix, reproduced_corr)


                # Simple Structure Count
                loadings_df = pd.DataFrame(loadings, index=df_efa_input.columns)
                simple_structure_count = count_simple_structure(loadings_df, simple_structure_threshold)

                # Max Factor Correlation (Phi) for oblique rotations
                max_abs_phi = 0.0
                if fa.phi_ is not None:
                    phi_lower_triangle = np.abs(np.tril(fa.phi_, k=-1))
                    if phi_lower_triangle.size > 0:
                         max_abs_phi = float(phi_lower_triangle.max())

                # ML-specific Fit Indices (ChiSq/df, RMSEA)
                chi_sq_per_df = np.nan
                rmsea = np.nan
                if method == "ml":
                    try:
                        model_stats = fa.get_model_stats()
                        if model_stats and model_stats.get("dof", 0) > 0:
                             chi_sq_per_df = model_stats["chi_square"] / model_stats["dof"]
                             rmsea = model_stats.get("rmsea", np.nan)
                    except Exception as ml_stat_error:
                        log_message(f"  Warning: Could not get ML stats for {method}+{rotation}: {ml_stat_error}")

                # Append results
                summary_data.append({
                    "Method": method,
                    "Rotation": rotation,
                    "Explained_Var(%)": total_explained_pct,
                    "RMSR": rmsr,
                    "ChiSq_per_DF": chi_sq_per_df, # Only for ML
                    "RMSEA": rmsea, # Only for ML
                    "Simple_Structure_Count": simple_structure_count,
                    "Max_Abs_Factor_Corr": max_abs_phi, # Only for oblique
                })

            except Exception as fit_error:
                warnings.warn(f"Model fitting failed for Method='{method}', Rotation='{rotation}': {fit_error}")
                summary_data.append({
                    "Method": method, "Rotation": rotation, "Explained_Var(%)": np.nan,
                    "RMSR": np.nan, "ChiSq_per_DF": np.nan, "RMSEA": np.nan,
                    "Simple_Structure_Count": np.nan, "Max_Abs_Factor_Corr": np.nan, "Error": str(fit_error)
                })


    log_message("\n--- Model Comparison Complete ---")
    return pd.DataFrame(summary_data)

In [ ]:
# =============================================================================
# Entry Point
# =============================================================================

# Define AGGREGATION_DEMOGRAPHIC_VARS globally or pass it if needed for fallback loading
AGGREGATION_DEMOGRAPHIC_VARS = [
    'B_COUNTRY_ALPHA', 'H_URBRURAL', 'Q260', 'X003R2', 'Q275R',
]

# I prefer jupyter so commenting out argparse for now
if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description="Compare EFA models with different methods and rotations.")
    # parser.add_argument("--n_factors", type=int, default=DEFAULT_N_FACTORS,
    #                     help=f"Number of factors to extract for comparison (default: {DEFAULT_N_FACTORS}).")
    # parser.add_argument("--loading_threshold", type=float, default=DEFAULT_LOADING_THRESHOLD,
    #                     help=f"Absolute loading threshold for simple structure calculation (default: {DEFAULT_LOADING_THRESHOLD:.2f}).")
    # parser.add_argument("--agg_file", default=str(AGGREGATED_DATA_PATH),
    #                     help=f"Path to the input aggregated CSV file (default: {AGGREGATED_DATA_PATH}).")
    # parser.add_argument("--var_file", default=str(VARIABLE_INFO_PATH),
    #                     help=f"Path to the variable info CSV file (default: {VARIABLE_INFO_PATH}).")
    # parser.add_argument("--out_dir", default=str(OUTPUT_DIR_ROOT),
    #                      help=f"Root directory for output (default: {OUTPUT_DIR_ROOT}). A date-stamped subfolder will be created.")
    # parser.add_argument("--out_prefix", default="",
    #                      help="Optional prefix for the date-stamped output subfolder.")

    # args = parser.parse_args()
    # DEFAULT_N_FACTORS
    # DEFAULT_LOADING_THRESHOLD
    # AGGREGATED_DATA_PATH
    # VARIABLE_INFO_PATH
    # OUTPUT_DIR_ROOT
    

    # --- Execute Pipeline ---
    try:
        # Create specific output directory for this run
        output_directory = ensure_output_dir(pathlib.Path(OUTPUT_DIR_ROOT), prefix="")

        # Load data
        df_efa_input = load_data_for_comparison(
            pathlib.Path(AGGREGATED_DATA_PATH), pathlib.Path(VARIABLE_INFO_PATH)
        )

        # Compare models
        comparison_results_df = compare_efa_models(
            df_efa_input=df_efa_input,
            n_factors=DEFAULT_N_FACTORS,
            methods=METHODS_TO_COMPARE,
            rotations=ROTATIONS_TO_COMPARE,
            simple_structure_threshold=DEFAULT_LOADING_THRESHOLD
        )

        # Sort results (by minimizing RMSR and maximizing Simple Structure)
        df_sorted = comparison_results_df.sort_values(
            by=["Simple_Structure_Count", "RMSR", "Explained_Var(%)"],
            ascending=[False, True, False] # High SS count, Low RMSR, High Explained Var
        ).reset_index(drop=True)

        # Print and save summary
        log_message("\n==== EFA Comparison Summary (Sorted) ====")
        float_fmt = "{:.3f}".format
        print(df_sorted.to_string(index=False, float_format=float_fmt, na_rep='NaN'))

        summary_csv_path = output_directory / f"efa_comparison_summary_n{DEFAULT_N_FACTORS}.csv"
        df_sorted.to_csv(summary_csv_path, index=False, float_format='%.4f')
        log_message(f"\nComparison results saved to: {summary_csv_path}")

    except FileNotFoundError as e:
        print(f"\nError: Input file not found. {e}", file=sys.stderr)
        sys.exit(1)
    except ValueError as e:
        print(f"\nError: Data validation or configuration issue. {e}", file=sys.stderr)
        sys.exit(1)
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}", file=sys.stderr)
        # import traceback
        # traceback.print_exc()
        sys.exit(1)

Output directory created/ensured: output\efa_comparison\20250506
--- Loading Data for Comparison ---
Loaded aggregated data: (2250, 102)
Identified 97 survey columns based on variable info file.

--- Comparing EFA Models (n_factors=5) ---
Data Suitability: Bartlett p=0, Overall KMO=0.943
Fitting Model: Method='minres', Rotation='promax'


c:\code\CultureEval2\.venv\lib\site-packages\factor_analyzer\utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


Fitting Model: Method='minres', Rotation='oblimin'
Fitting Model: Method='minres', Rotation='varimax'
Fitting Model: Method='principal', Rotation='promax'
Fitting Model: Method='principal', Rotation='oblimin'
Fitting Model: Method='principal', Rotation='varimax'
Fitting Model: Method='ml', Rotation='promax'
Fitting Model: Method='ml', Rotation='oblimin'
Fitting Model: Method='ml', Rotation='varimax'

--- Model Comparison Complete ---

==== EFA Comparison Summary (Sorted) ====
   Method Rotation  Explained_Var(%)  RMSR  ChiSq_per_DF  RMSEA  Simple_Structure_Count  Max_Abs_Factor_Corr
principal   promax            35.224 0.074           NaN    NaN                      58                0.359
   minres   promax            31.895 0.077           NaN    NaN                      55                0.419
principal  oblimin            35.269 0.082           NaN    NaN                      52                0.397
       ml   promax            31.833 0.087           NaN    NaN                    

```python
Fitting Model: Method='minres', Rotation='oblimin'
Fitting Model: Method='minres', Rotation='varimax'
Fitting Model: Method='principal', Rotation='promax'
Fitting Model: Method='principal', Rotation='oblimin'
Fitting Model: Method='principal', Rotation='varimax'
Fitting Model: Method='ml', Rotation='promax'
  Warning: Could not get ML stats for ml+promax: 'FactorAnalyzer' object has no attribute 'get_model_stats'
Fitting Model: Method='ml', Rotation='oblimin'
  Warning: Could not get ML stats for ml+oblimin: 'FactorAnalyzer' object has no attribute 'get_model_stats'
Fitting Model: Method='ml', Rotation='varimax'
  Warning: Could not get ML stats for ml+varimax: 'FactorAnalyzer' object has no attribute 'get_model_stats'

--- Model Comparison Complete ---

==== EFA Comparison Summary (Sorted) ====
   Method Rotation  Explained_Var(%)  RMSR  ChiSq_per_DF  RMSEA  Simple_Structure_Count  Max_Abs_Factor_Corr
principal   promax            35.224 0.074           NaN    NaN                      58                0.359
   minres   promax            31.895 0.077           NaN    NaN                      55                0.419
principal  oblimin            35.269 0.082           NaN    NaN                      52                0.397
       ml   promax            31.833 0.087           NaN    NaN                      51                0.503
   minres  oblimin            31.839 0.090           NaN    NaN                      51                0.502
       ml  oblimin            31.852 0.094           NaN    NaN                      50                0.565
       ml  varimax            34.847 0.046           NaN    NaN                      48                0.000
   minres  varimax            35.250 0.045           NaN    NaN                      46                0.000
principal  varimax            38.155 0.047           NaN    NaN                      46                0.000

Comparison results saved to: output\efa_comparison\20250506\efa_comparison_summary_n5.csv
```